In [3]:
import os
str1 = '交易明细信息'
file_list = []
os.chdir(r"E:\7.3徐州网赌\Q9案8.2反馈数据")
for path_name, dir, files_name in os.walk(os.getcwd()):
    for file in files_name:
        if str1 in file:
            file_list.append(os.path.join(path_name, file))
os.chdir(r"E:\7.3徐州网赌\Q9案8.4反馈数据（64个账户）")
for path_name, dir, files_name in os.walk(os.getcwd()):
    for file in files_name:
        if str1 in file:
            file_list.append(os.path.join(path_name, file))
            
print(len(file_list))
file_list

22


['E:\\7.3徐州网赌\\Q9案8.2反馈数据\\上海浦东发展银行股份有限公司交易明细信息.csv',
 'E:\\7.3徐州网赌\\Q9案8.2反馈数据\\中国农业银行交易明细信息.csv',
 'E:\\7.3徐州网赌\\Q9案8.2反馈数据\\中国建设银行交易明细信息.csv',
 'E:\\7.3徐州网赌\\Q9案8.2反馈数据\\中国民生银行股份有限公司交易明细信息.csv',
 'E:\\7.3徐州网赌\\Q9案8.2反馈数据\\中国邮政储蓄银行有限责任公司交易明细信息.csv',
 'E:\\7.3徐州网赌\\Q9案8.2反馈数据\\中国银行交易明细信息.csv',
 'E:\\7.3徐州网赌\\Q9案8.2反馈数据\\交通银行股份有限公司交易明细信息.csv',
 'E:\\7.3徐州网赌\\Q9案8.2反馈数据\\兴业银行股份有限公司交易明细信息.csv',
 'E:\\7.3徐州网赌\\Q9案8.2反馈数据\\平安银行交易明细信息.csv',
 'E:\\7.3徐州网赌\\Q9案8.2反馈数据\\招商银行股份有限公司交易明细信息.csv',
 'E:\\7.3徐州网赌\\Q9案8.2反馈数据\\福建省农村信用社联合社交易明细信息.csv',
 'E:\\7.3徐州网赌\\Q9案8.4反馈数据（64个账户）\\上海浦东发展银行股份有限公司交易明细信息.csv',
 'E:\\7.3徐州网赌\\Q9案8.4反馈数据（64个账户）\\中信银行交易明细信息.csv',
 'E:\\7.3徐州网赌\\Q9案8.4反馈数据（64个账户）\\中国光大银行股份有限公司交易明细信息.csv',
 'E:\\7.3徐州网赌\\Q9案8.4反馈数据（64个账户）\\中国建设银行交易明细信息.csv',
 'E:\\7.3徐州网赌\\Q9案8.4反馈数据（64个账户）\\中国民生银行股份有限公司交易明细信息.csv',
 'E:\\7.3徐州网赌\\Q9案8.4反馈数据（64个账户）\\中国银行交易明细信息.csv',
 'E:\\7.3徐州网赌\\Q9案8.4反馈数据（64个账户）\\交通银行股份有限公司交易明细信息.csv',
 'E:\\7.3徐州网赌\\Q9案8.4反馈数据（64个账户）\\兴业银行股份有限公司交易明细信息.csv',
 'E:\\7.3徐州

In [4]:
import pandas as pd
df_group = pd.DataFrame(columns = ['Path','Columns','False','True'])
type1 = '.xls'
type2 = '.csv'
for i in range(len(file_list)):
#     print(i)
    if type1 in file_list[i]:
        df = pd.read_excel(file_list[i], encoding = "gbk")
    elif type2 in file_list[i]:
        df = pd.read_csv(file_list[i], engine = "python")
    col_name = df.columns
    bool_S = df.duplicated().value_counts()
    if len(bool_S)==1:
        df_group.loc[i] = {'Path':file_list[i],'Columns':tuple(col_name),'False':bool_S[0],'True':0}
    elif len(bool_S)==2:
        df_group.loc[i] = {'Path':file_list[i],'Columns':tuple(col_name),'False':bool_S[0],'True':bool_S[1]}
    
grouped = df_group.groupby('Columns')
groupname_list = list(grouped.size().index)
print('以下'+str(len(groupname_list))+'种格式数据分类，保留流水、账户类别')
print(grouped.size())

以下1种格式数据分类，保留流水、账户类别
Columns
(交易卡号\t, 交易账号\t, 查询反馈结果原因\t, 交易户名\t, 交易证件号码\t, 交易时间\t, 交易金额\t, 交易余额\t, 收付标志\t, 交易对手账卡号\t, 现金标志\t, 对手户名\t, 对手身份证号\t, 对手开户银行\t, 摘要说明\t, 交易币种\t, 交易网点名称\t, 交易发生地\t, 交易是否成功\t, 传票号\t, IP地址\t, MAC地址\t, 对手交易余额\t, 交易流水号\t, 日志号\t, 凭证种类\t, 凭证号\t, 交易柜员号\t, 备注\t, 交易类型\t)    22
dtype: int64


输出流水类、账户类文件，每个文件内重复数据情况(汇总在df_group)

In [8]:
import numpy as np
import re
import datetime

df_忽略行 = pd.DataFrame()
df_冲正 = pd.DataFrame()
all_data = pd.DataFrame()
# all_account = pd.DataFrame()

group_流水 = grouped.get_group(groupname_list[0])
for i in range(len(file_list)):
    if file_list[i] in list(group_流水['Path']):  #查控平台流水
        if type1 in file_list[i]:
            df = pd.read_excel(file_list[i],encoding = "gbk")
        elif type2 in file_list[i]:
            df = pd.read_csv(file_list[i], engine="python")
        df.columns = df.columns.str.strip()
        col_name = df.columns.tolist()
        
        df['交易场所'] = df['凭证号']
        df['渠道'] = df['凭证号']
        
        df['path'] = file_list[i]
        df['序号'] = df.index
        df = df.applymap(lambda x: re.sub('\t','',str(x)))
        df['交易卡号'] = df['交易卡号'].map(lambda x: np.NaN if str(x)=='' else x)
        df['交易卡号'][df['交易卡号'].isnull()]=df['交易账号'][df['交易卡号'].isnull()] # 账号填充卡号空白
        df_忽略行 = df_忽略行.append(df[(df['交易金额']=='') | (df['交易卡号']=='')])
        df = df.drop(df[(df['交易金额']=='') | (df['交易卡号']=='')].index)
        df['交易金额'] = df['交易金额'].map(lambda x: float(x))  #交易金额指定浮点型
        df['交易余额'] = df['交易余额'].map(lambda x: np.NaN if str(x)=='' else x)
        df['交易余额'] = df['交易余额'].map(lambda x: float(x))  #交易余额指定浮点型
        df = df.drop_duplicates(subset = col_name[0:16],keep = 'last')  #去除重复
        df = df.sort_values(by = ['交易卡号','交易时间','交易金额'],ascending=[True,True,False]).reset_index().iloc[:,1:]
        print(str(i)+'. 单表去重后记录数:'+str(df.shape[0]))
        df_负 = df[df['交易金额']<0]
        for j in range(0,len(df_负)):
            df_正 = df[(df['交易金额']== abs(df_负['交易金额'].iloc[j])) & \
                (df['收付标志']== df_负['收付标志'].iloc[j]) & \
                (df['交易时间']<= df_负['交易时间'].iloc[j]) & \
                (df['交易卡号']== df_负['交易卡号'].iloc[j]) & \
                (
                 (df['交易对手账卡号']== df_负['交易对手账卡号'].iloc[j]) | \
                 ((df['交易对手账卡号'] == '') & (df_负['交易对手账卡号'].iloc[j] == '')) | \
                 ((df['交易对手账卡号'] != '') & (df_负['交易对手账卡号'].iloc[j] == '')) | \
                 ((df['交易对手账卡号'] == '') & (df_负['交易对手账卡号'].iloc[j] != '')) | \
                 ((df['交易对手账卡号'] != '') & (df['交易卡号']==df_负['交易对手账卡号'].iloc[j]))
                )
                ]
            d_labels = [df_正.index.max(),df_负.index[j]]  #冲正匹配记录的索引
            if len(df_正)>0:
                df_冲正 = df_冲正.append(df.loc[d_labels])
                df = df.drop(d_labels)
                print(d_labels)
            elif len(df_正)==0:
                print(d_labels)
        print('剔除冲正后保留记录数:'+str(df.shape[0]))
        all_data = all_data.append(df)


all_data.to_csv("E:\\7.3徐州网赌\\流水合并剔除冲正.csv",index = None)
print(all_data.shape)

with pd.ExcelWriter("E:\\7.3徐州网赌\\剔除行.xlsx") as writer:
    df_冲正.to_excel(writer, sheet_name='冲正全',index = None)
    df_忽略行.to_excel(writer, sheet_name='忽略行',index = None)

print(df_忽略行.shape)
print(df_冲正.shape)


0. 单表去重后记录数:2283
剔除冲正后保留记录数:2283
1. 单表去重后记录数:41119
剔除冲正后保留记录数:41119
2. 单表去重后记录数:47865
[1889, 1890]
[1892, 1950]
[1959, 1998]
[2224, 2249]
[2300, 2305]
[nan, 2538]
[2549, 2552]
[2659, 2661]
[2677, 2679]
[2941, 2942]
[2952, 2953]
[nan, 2991]
[3180, 3181]
[3233, 3234]
[3527, 3530]
[3526, 3531]
[3528, 3537]
[3529, 3538]
[3557, 3560]
[3556, 3567]
[3789, 3795]
[3792, 3796]
[4007, 4008]
[4026, 4027]
[4266, 4267]
[4272, 4275]
[4296, 4297]
[4344, 4348]
[4343, 4353]
[nan, 4450]
[4469, 4470]
[4462, 4478]
[4464, 4483]
[4699, 4700]
[4719, 4720]
[4718, 4721]
[4717, 4722]
[4840, 4841]
[4984, 4985]
[5290, 5291]
[5467, 5476]
[6039, 6040]
[6041, 6042]
[6321, 6322]
[nan, 6521]
[nan, 6534]
[6965, 6966]
[8373, 8375]
[8792, 8793]
[nan, 9286]
[9291, 9324]
[9325, 9326]
[9327, 9332]
[10014, 10015]
[10215, 10216]
[nan, 10637]
[nan, 10663]
[nan, 10664]
[10647, 10665]
[10840, 10841]
[10839, 10842]
[10838, 10843]
[nan, 11349]
[11950, 11951]
[11953, 11956]
[11960, 11965]
[11966, 11972]
[11975, 11979]
[12010, 12011]

D:\Users\ThinkPad\anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


6. 单表去重后记录数:5822
剔除冲正后保留记录数:5822
7. 单表去重后记录数:558
[430, 431]
剔除冲正后保留记录数:556
8. 单表去重后记录数:3560
[247, 248]
[483, 484]
[849, 850]
[1610, 1611]
剔除冲正后保留记录数:3552
9. 单表去重后记录数:4071
剔除冲正后保留记录数:4071
10. 单表去重后记录数:1513
剔除冲正后保留记录数:1513
11. 单表去重后记录数:1972
剔除冲正后保留记录数:1972
12. 单表去重后记录数:4739
[1759, 1760]
[2706, 2707]
[2708, 2709]
[2710, 2711]
[2744, 2745]
[2746, 2747]
[3363, 3364]
剔除冲正后保留记录数:4725
13. 单表去重后记录数:933
[44, 45]
剔除冲正后保留记录数:931
14. 单表去重后记录数:15320
[1300, 1301]
[2157, 2158]
[2159, 2160]
[2628, 2630]
[2627, 2635]
[2626, 2636]
[2629, 2637]
[2731, 2732]
[2783, 2784]
[2785, 2787]
[nan, 2893]
[4212, 4213]
[4305, 4306]
[4307, 4308]
[4367, 4368]
[nan, 4451]
[nan, 4454]
[4545, 4546]
[4586, 4593]
[4584, 4594]
[4590, 4596]
[4592, 4597]
[4647, 4650]
[nan, 4707]
[4748, 4749]
[4769, 4770]
[4771, 4772]
[4787, 4788]
[4874, 4875]
[5092, 5093]
[5094, 5095]
[5096, 5098]
[5097, 5099]
[5323, 5324]
[5558, 5559]
[5714, 5731]
[5930, 5948]
[5996, 6012]
[6154, 6155]
[6252, 6267]
[6333, 6334]
[6370, 6371]
[6404, 6405]
[6427

In [9]:
account2 = pd.read_excel('E:\\7.3徐州网赌\\银行卡人员信息整理(提供给公司).xls',sheet_name = '去重',encoding = 'gbk')
dfs2 = pd.merge(all_data,account2.drop_duplicates(subset='银行卡号'),how = 'left',left_on='交易卡号',right_on='银行卡号')
dfs2['交易户名'][dfs2['姓名'].notnull()] = dfs2['姓名'][dfs2['姓名'].notnull()]

df = dfs2.iloc[:,:34]
df.columns = all_data.columns
print(df.shape)

(166511, 34)


D:\Users\ThinkPad\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:

df_余额 = pd.DataFrame(columns = ['交易卡号','交易户名','交易时间','交易余额'])
df_S1 = df.groupby('交易卡号').size().reset_index()
df_S2 = pd.DataFrame(columns = ['交易卡号','Size','False','True','最早时间','最晚时间'])
for i in df_S1.index:
    df_every = df.groupby('交易卡号').get_group(df_S1['交易卡号'][i])
    bool_S = df_every.duplicated(subset = col_name[0:16],keep = 'last').value_counts()
    if len(bool_S)==1:
        df_S2.loc[i] = {'交易卡号':df_every['交易卡号'].iloc[0],'Size':df_every.shape[0],'False':bool_S[0],'True':0,'最早时间':df_every['交易时间'].min(),'最晚时间':df_every['交易时间'].max()}
    elif len(bool_S)==2:
        df_S2.loc[i] = {'交易卡号':df_every['交易卡号'].iloc[0],'Size':df_every.shape[0],'False':bool_S[0],'True':bool_S[1],'最早时间':df_every['交易时间'].min(),'最晚时间':df_every['交易时间'].max()}
    df_余额.loc[i] = df_every[['交易卡号','交易户名','交易时间','交易余额']].loc[df_every.index.max()]

with pd.ExcelWriter("E:\\7.3徐州网赌\\单卡统计.xlsx") as writer:
    df_S2.to_excel(writer,sheet_name = '单卡去重统计',index = None)
    df_余额.to_excel(writer,sheet_name = '单卡最后一笔交易余额',index = None)
# print(df_S1)
print(df_S2)
print(df.duplicated(subset = col_name[0:16]).value_counts())
df = df.drop_duplicates(subset = col_name[0:16],keep = 'last')
df.to_csv("E:\\7.3徐州网赌\\流水户名.csv",index = None)

                   交易卡号  Size False True                 最早时间  \
0   6013827507000949885  1006  1006    0  2020-01-12 06:25:35   
1      6214832188976661   983   983    0  2020-01-11 10:46:02   
2      6214832199914149   733   733    0  2020-01-01 02:19:23   
3      6214832509679143  1420  1420    0  2020-01-01 10:22:54   
4      6214834362078645  1046  1046    0  2020-01-04 07:00:42   
..                  ...   ...   ...  ...                  ...   
87  6230580000269674458   576   576    0  2020-01-04 09:10:18   
88  6236681500001052181  1148  1148    0  2020-03-18 09:36:30   
89  6236682000007113250   738   738    0  2020-01-01 15:32:54   
90  6236682000013335798  1836  1836    0  2020-01-01 08:30:22   
91  6236682990001978825   989   989    0  2020-01-01 09:05:23   

                   最晚时间  
0   2020-07-30 09:24:28  
1   2020-07-16 02:47:39  
2   2020-07-30 20:47:32  
3   2020-05-18 11:40:32  
4   2020-08-01 02:32:51  
..                  ...  
87  2020-07-06 15:06:40  
88  2020-07

In [11]:
primary_card = df_S2.iloc[:,0].tolist()
print(primary_card[0])
box1 = df[df['交易对手账卡号'].isin(primary_card)]
col_2 = box1.columns.tolist()+['付款方','收款方']
box2 = pd.DataFrame(box1,columns = col_2)
print(box2.shape)
box2['付款方'][box2['收付标志']=='出'] = box2['交易卡号'][box2['收付标志']=='出']
box2['收款方'][box2['收付标志']=='出'] = box2['交易对手账卡号'][box2['收付标志']=='出']
box2['付款方'][box2['收付标志']=='进'] = box2['交易对手账卡号'][box2['收付标志']=='进']
box2['收款方'][box2['收付标志']=='进'] = box2['交易卡号'][box2['收付标志']=='进']

box_group = box2.groupby(['付款方','收款方'])
box_group.size().to_excel("E:\\7.3徐州网赌\\box.xlsx")
print(len(box_group.size().reset_index()))

6013827507000949885
(1983, 36)
34


D:\Users\ThinkPad\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
D:\Users\ThinkPad\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
D:\Users\ThinkPad\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
D:\Users\ThinkPad\anaconda3\lib\site-packages\ipy

In [12]:

# 前提是调取的数据以日为最小单位，实际调取至今的话“至今”最小单位时分秒，有误差，例如6217003800028006371
# 查控平台数据收付标志有误，例如6230720210512159709 曾路恒

box_AB = pd.DataFrame()
df_AB = df
for i in range(0,len(box_group.size().reset_index())):
        box_every = box_group.get_group(box_group.size().index[i])
        box_every['日期'] = box_every['交易时间'].str[0:10]

        if len(box_every.drop_duplicates(['交易卡号']))==2:
            if box_every[box_every['收付标志']=='出']['日期'].min()>box_every[box_every['收付标志']=='进']['日期'].min():
                box_every = box_every[box_every['日期']>=box_every[box_every['收付标志']=='出']['日期'].min()]
            else:
                box_every = box_every[box_every['日期']>=box_every[box_every['收付标志']=='进']['日期'].min()]
            if len(box_every.drop_duplicates(['交易卡号']))==2:
                if box_every[box_every['收付标志']=='出']['日期'].max()<box_every[box_every['收付标志']=='进']['日期'].max():
                    box_every = box_every[box_every['日期']<=box_every[box_every['收付标志']=='出']['日期'].max()]
                else:
                    box_every = box_every[box_every['日期']<=box_every[box_every['收付标志']=='进']['日期'].max()]
                box_AB = box_AB.append(box_every)
                df_AB = df_AB.drop(box_every[box_every['收付标志']=='出'].index)
                print(str(i)+'去重后保留'+str(df_AB.shape[0]))
            else:
                print(str(i)+'无交集')
        else:
                print(i)
df_AB.to_csv("E:\\7.3徐州网赌\\流水去重后.csv",index = None)
box_AB.sort_values(by = ['付款方','收款方','交易时间','收付标志'],ascending=True).to_excel("E:\\7.3徐州网赌\\流水去重匹配.xlsx") 
print(df_AB.shape)
print(box_AB.shape)
print('Done!')

D:\Users\ThinkPad\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


0
1
2去重后保留166387
3去重后保留166301
4
5去重后保留166104
6
7去重后保留166090
8
9去重后保留166039
10去重后保留166038
11去重后保留166036
12去重后保留166035
13
14
15
16去重后保留165782
17
18去重后保留165766
19
20去重后保留165761
21去重后保留165758
22
23
24
25
26去重后保留165755
27
28
29
30去重后保留165746
31
32
33
(165746, 34)
(1538, 37)
Done!
